In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import *

In [2]:
engine = create_engine("sqlite:///data/movies-db.sqlite")
conn = engine.connect()

In [ ]:
df = pd.read_csv("data/genome-scores.csv")
df2 = pd.read_csv("data/genome-tags.csv")
df3 = pd.read_csv("data/links.csv")
df4 = pd.read_csv("data/movies.csv")
df5 = pd.read_csv("data/ratings.csv")

In [4]:
tags_df = df.pivot(index='movieId', columns='tagId', values='relevance')
tags_df.to_sql('tags_rel', con=conn)

In [5]:
tags_df.head()

tagId,1,2,3,4,5,6,7,8,9,10,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.02900,0.02375,0.05425,0.06875,0.16000,0.19525,0.07600,0.25200,0.22750,0.02400,...,0.03775,0.02250,0.04075,0.03175,0.12950,0.04550,0.02000,0.03850,0.09125,0.02225
2,0.03625,0.03625,0.08275,0.08175,0.10200,0.06900,0.05775,0.10100,0.08225,0.05250,...,0.04775,0.02050,0.01650,0.02450,0.13050,0.02700,0.01825,0.01225,0.09925,0.01850
3,0.04150,0.04950,0.03000,0.09525,0.04525,0.05925,0.04000,0.14150,0.04075,0.03200,...,0.05800,0.02375,0.03550,0.02125,0.12775,0.03250,0.01625,0.02125,0.09525,0.01750
4,0.03350,0.03675,0.04275,0.02625,0.05250,0.03025,0.02425,0.07475,0.03750,0.02400,...,0.04900,0.03275,0.02125,0.03675,0.15925,0.05225,0.01500,0.01600,0.09175,0.01500
5,0.04050,0.05175,0.03600,0.04625,0.05500,0.08000,0.02150,0.07375,0.02825,0.02375,...,0.05375,0.02625,0.02050,0.02125,0.17725,0.02050,0.01500,0.01550,0.08875,0.01575


In [ ]:
movies_df = df4.drop(columns='genres')

In [ ]:
ratings_df = df5.groupby('movieId').mean()
ratings = ratings_df.drop(columns = ['userId', 'timestamp'])
ratings.head()
ratings.to_csv('data/ratings_df.csv')

In [ ]:
movies_ratings_df = pd.merge(movies_df, ratings, how='outer', on='movieId')

In [ ]:
movies_ratings_df.head()

In [ ]:
full_df = pd.merge(movies_ratings_df, tags_df, how='outer', on='movieId')

In [ ]:
full_df.to_sql('movies', con=conn)

In [ ]:
selections_df = full_df.drop(columns = ['movieId','title'])

In [ ]:
selections_df.head()

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=1000)

In [ ]:
kmeans.fit(selections_df)

In [ ]:
predicted_clusters = kmeans.predict(selections_df)
print(predicted_clusters)

In [ ]:
clusters = pd.DataFrame(predicted_clusters)
full_df['clusters']=clusters

In [ ]:
full_df.head()

In [ ]:
inspector = inspect(engine)
tables = inspector.get_table_names()
tables

In [ ]:
second_cluster = pd.read_sql("SELECT * FROM movies Where clusters = '100' ", con=conn)

In [ ]:
second_cluster.head()

In [ ]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10)

In [ ]:
knn.fit(selections_df)
knn.kneighbors([selections_df.iloc[1]], return_distance = False)

In [ ]:
full_df.iloc[1]

In [ ]:
selections2_df = 